In [1]:
# Imports
from sunpy.net import Fido, attrs as a
import astropy.units as u
import os
import sys


In [ ]:
# Ensures that the user enters an input
def get_user_input(prompt):
  """
  Prompts the user for input and returns the sanitized input.
  """
  while True:
    user_input = input(prompt)
    print(user_input)
    if user_input:
      return user_input.strip()
    else:
      print('No Input!')

def apply_time_window(result, minutes=None):
  if minutes:
    h = round(minutes / 60)
    m = minutes % 60
    window = f'{h:02d}:{m:02d}'
  else:
    window = get_user_input("Please enter a time range around the STEREO image that you would like to search (HH:MM):")
    
  start = str(result[0]["Start Time"][0])
  date = start.split(" ")[0]
  tstring = start.split(" ")[1]

  w_hour = int(window.split(':')[0])
  w_min  = int(window.split(':')[1])

  t_hour = int(tstring.split(':')[0])
  t_min  = int(tstring.split(':')[1])

  # Calculate new start time with hour adjustments
  new_start_hour = t_hour - w_hour
  new_start_min = t_min - w_min

  # Adjust for minutes going below 0
  if new_start_min < 0:
      new_start_hour -= 1
      new_start_min += 60

  # Adjust for minutes going above 60
  elif new_start_min >= 60:
      new_start_hour += 1
      new_start_min -= 60

  # Ensure hour is within 0-23 range
  new_start_hour = (new_start_hour + 24) % 24 

  new_start = f'{date} {new_start_hour:02d}:{new_start_min:02d}' 

  # Calculate new end time with hour adjustments
  new_end_hour = t_hour + w_hour
  new_end_min = t_min + w_min

  # Adjust for minutes going below 0
  if new_end_min < 0:
      new_end_hour -= 1
      new_end_min += 60

  # Adjust for minutes going above 60
  elif new_end_min >= 60:
      new_end_hour += 1
      new_end_min -= 60

  # Ensure hour is within 0-23 range
  new_end_hour = (new_end_hour + 24) % 24

  new_end = f'{date} {new_end_hour:02d}:{new_end_min:02d}'

  time = a.Time(new_start, new_end)

  return time

def approx_stereo(stereo_result, wav):
  n_aia = 0
  mins = 0
  while n_aia == 0:
    mins += 10
    print(f'No AIA in stereo range! \nAttempting search in {mins}-minute window...\n')
    time = apply_time_window(stereo_result, mins)
    aia_result = Fido.search(time, a.Instrument.aia, a.Sample(1*u.minute), a.Physobs('Intensity'), wav)
    print("AIA results within STEREO timeframe:")
    print(aia_result.show('Start Time', 'End Time', 'Source'))
    n_aia = aia_result.__dict__['_numfile']
  
  return aia_result

def get_instrument_query(instrument_input):
    """Prompts the user for an instrument and returns a valid Fido query."""

    valid_instruments = {  # Define valid instruments and their Fido-compatible names
        "aia": a.Instrument('AIA'),
        "hmi": a.Instrument('HMI'),
        "goes": a.Instrument('GOES'),  # Or a.Instrument('GOES', satellite=15) etc. if needed
        "stereo": a.Instrument('SECCHI'),
        "stereo_a": a.Instrument('SECCHI', spacecraft="STEREO_A"), # Example for specific spacecraft
        "stereo_b": a.Instrument('SECCHI', spacecraft="STEREO_A"), # Example for specific spacecraft
        "rhses": a.Instrument('RHESSI'),
        # ... add more instruments as needed
    }

    while True:
        if instrument_input in valid_instruments:
            return valid_instruments[instrument_input]
        elif instrument_input == "list": # Allow user to see the options
          print("Valid instruments are: ", list(valid_instruments.keys()))
        else:
            print("Invalid instrument. Please choose from the following or type 'list':", list(valid_instruments.keys()))



debug = False

# Sets a time range
while True:
  try:
    if debug:
      start_time = "2012-07-19T10:00"
      end_time = "2012-07-19T11:00"
    else:
      # Prompt user for desired time range
      # Example: 2012-07-19T10:00, 2012-07-19T11:00
      start_time = get_user_input("Enter the start time (eg. 2012-07-19T10:00): ")
      end_time = get_user_input("Enter the end time (eg. 2012-07-19T11:00): ")

    # Convert user input to Fido.Time objects
    time = a.Time(start_time, end_time)
    break
  except:
    if start_time == 'exit': sys.exit(1)
    print('Time range not valid. Try again!')

# Sets the target wavelengths
while True:
  try:
    if debug:
      wavelength = '171'
    else:
      # Prompt user to select desired AIA wavelengths (comma separated)
      wavelength = get_user_input("Enter target wavelength eg. 171, 195, 304 (Angstroms): ")
    
    wavelength = a.Wavelength(float(wavelength)*u.angstrom)
    break
  except:
    if wavelength == 'exit': sys.exit(1)
    print('Wavelength selection not valid. Try again!')

instrument = None
# Sets the target instrument
while True:
  try:
    if debug:
      user_instrument = 'aia'
      instrument = a.Instrument.aia
    else:
      # Prompt user to select desired AIA wavelengths (comma separated)
      user_instrument = get_user_input("Enter target instrument (eg. 'aia', 'stereo_a', 'stereo_b'): ")
      instrument = get_instrument_query(user_instrument)
    
    break
  except:
    if instrument == 'exit': sys.exit(1)
    print('Instrument selection not valid. Try again!')


# Containers for later retrieval of results
aia_results = []
stereo_results = []
results = []

aia_wav = wavelength
stereo_wav = wavelength

# If the wavelength is 195, search for 193 instead
if stereo_wav == a.Wavelength(float(193)*u.angstrom):
  stereo_wav = a.Wavelength(float(195)*u.angstrom)

# Search for STEREO data first
stereo_result = Fido.search(time, a.Instrument.secchi, a.Sample(1*u.minute), a.Physobs('Intensity'), stereo_wav)
print("STEREO results:")
print(stereo_result.show('Start Time', 'End Time', 'Source'))

n_stereo = stereo_result.__dict__['_numfile']

# Saves the results for next cell
stereo_results.append(stereo_result)

# Saves the results to local .txt file for easy viewing (notebook may truncate output)
directory_path = 'fido_results/'
if not os.path.exists(directory_path):
    os.makedirs(directory_path)
file = open(f'{directory_path}{start_time}_{end_time}_{user_instrument}_{wavelength.max.value}.txt', 'w')
file.write(str(stereo_result))
file.close()


2012-07-19T10:00
2012-07-19T11:00
131
aia
Instrument selection not valid. Try again!
aia
Instrument selection not valid. Try again!
exit
Instrument selection not valid. Try again!
0
Instrument selection not valid. Try again!
escape
Instrument selection not valid. Try again!

No Input!

No Input!

No Input!

No Input!

No Input!

No Input!
exit
Instrument selection not valid. Try again!


In [ ]:
# Allows the user to select which searched event to download

debug = False
if not FITS_DIR:
  FITS_DIR = './observations/' # <-- Change this if you would like a custom download location
  directory_path = FITS_DIR
  if not os.path.exists(directory_path):
    os.makedirs(directory_path)

  for w, wav in enumerate(wavs):
      while True:
        try:
          if debug:
            aia_selection = '-1'
            stereo_selection = '-1'
          else:
            # Allow user to select results to download
            aia_selection = get_user_input("Enter the index of the AIA result to download (or -1 to skip): ")
            stereo_selection = get_user_input("Enter the index of the STEREO result to download (or -1 to skip): ")

          if int(aia_selection) > -1:
            aia_sel = aia_results[w][0, int(aia_selection)]
            download = Fido.fetch(aia_sel, path=FITS_DIR)
            print("Download errors (AIA):", download.errors)

          # Download selected STEREO data
          if int(stereo_selection) > -1:
            stereo_sel = stereo_results[w][0, int(stereo_selection)]
            download = Fido.fetch(stereo_sel, path=FITS_DIR)
            print("Download errors (STEREO):", download.errors)

          break
        except:
          if aia_selection == 'exit': sys.exit(1)
          print('Selection not valid. Try again!')
